In [1]:
%pip install pandas
%pip install neattext
import nltk
nltk.download('punkt')
from nltk import tokenize
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
import neattext as nt
import neattext.functions as nfx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 KB 12.8 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# data_path = '/Users/charansaisadla/Desktop/ub classes/SEM2/NLP/data project/'
ctrl = pd.read_csv('Ctrl_generation.csv')
gpt = pd.read_csv('GPT_generation.csv')
gpt2 = pd.read_csv('gpt2_generate.csv')
grover = pd.read_csv('Grover_Generate.csv')
xlm = pd.read_csv('xlm_generation.csv')
xlnet = pd.read_csv('xlnet_generation.csv')
pplm = pd.read_csv('pplm.csv')
human = pd.read_csv('human.csv')
fair = pd.read_csv('generation_em_lm.csv')
gpt3=pd.read_csv('GPT3_curieData.csv')
instructgpt=pd.read_csv('GPT3_davinciData.csv')
titles = pd.read_csv('Titles.csv')
human = human['Articles']
human = pd.DataFrame({'Generation': list(human)})

xlm.head()




,Unnamed: 0,Title,Generation
0,0,Latest Headlines on CNN Business,"['Latest Headlines on CNN Business, tuesday an..."
1,1,China wants to take a victory lap over its han...,['China wants to take a victory lap over its h...
2,2,Coronavirus disinformation creates challenges ...,['Coronavirus disinformation creates challenge...
3,3,China coronavirus: Eating wild animals made il...,"[""China coronavirus: Eating wild animals made ..."
4,4,China's economy could shrink for the first tim...,['China\'s economy could shrink for the first ...


In [3]:
def combine_title_generation(row):
    return row['Title'] + ' ' + row['Generation']

grover['Generation'] = grover.apply(combine_title_generation, axis=1)
gpt3['Generation'] = gpt3.apply(combine_title_generation, axis=1)
instructgpt['Generation'] = instructgpt.apply(combine_title_generation, axis=1)


In [4]:
ctrl = ctrl[['Generation']]
gpt = gpt[['Generation']]
gpt2 = gpt2[['Generation']]
grover = grover[['Generation']]
xlm = xlm[['Generation']]
xlnet = xlnet[['Generation']]
pplm = pplm[['Generation']]
human = human[['Generation']]
fair = grover[['Generation']]
gpt3=gpt3[['Generation']]
instructgpt=instructgpt[['Generation']]


In [5]:
ctrl['label'] = 'ctrl'
gpt['label'] = 'gpt'   
gpt2['label'] = 'gpt2' 
grover['label'] = 'grover' 
xlm['label'] = 'xlm'       
xlnet['label'] = 'xlnet'   
pplm['label'] = 'pplm'    
human['label'] = 'human'  
fair['label'] = 'fair'
gpt3['label']='gpt3'
instructgpt['label']='instructgpt'




<ipython-input-5-2598760b056e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ctrl['label'] = 'ctrl'
<ipython-input-5-2598760b056e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gpt['label'] = 'gpt'
<ipython-input-5-2598760b056e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

In [6]:
def clean_individual_texts(data):
    cleaned_texts = []
    for text in data['Generation']:
        # apply preprocessing steps
        text = nfx.remove_special_characters(text)
        text = nfx.remove_punctuations(text)
        text = nfx.remove_emojis(text)
        text = nfx.remove_html_tags(text)
        text=text.lower()
        
        cleaned_texts.append(text)

    return pd.DataFrame({'Generation': cleaned_texts,'label': data['label']})
ctrl = clean_individual_texts(ctrl)
gpt = clean_individual_texts(gpt)
gpt2 = clean_individual_texts(gpt2)
grover = clean_individual_texts(grover)
xlm = clean_individual_texts(xlm)
xlnet = clean_individual_texts(xlnet)
pplm = clean_individual_texts(pplm)
human = clean_individual_texts(human)
fair = clean_individual_texts(grover)
gpt3=clean_individual_texts(gpt3)
instructgpt=clean_individual_texts(instructgpt)
gpt.tail()

,Generation,label
1061,how much of your body is your own n she didnt...,gpt
1062,how do you keep a space station clean n she s...,gpt
1063,the city where you pay a years rent up front ...,gpt
1064,the bbc news app gives you the best of bbc new...,gpt
1065,learn how the bbc is working to strengthen tru...,gpt


In [7]:
def stat(data):
    word = []
    sent = []
    
    for i in data:
        word.append(len(word_tokenize(i)))
        sent.append(len(tokenize.sent_tokenize(i)))
        
    final_list = [x for x in word if (x > np.mean(word) - 2 * np.std(word))]
    new_word = [x for x in final_list if (x < np.mean(word) + 2 * np.std(word))]
    
    final_list = [x for x in sent if (x > np.mean(sent) - 2 * np.std(sent))]
    new_sent = [x for x in final_list if (x < np.mean(sent) + 2 * np.std(sent))]
        
    return pd.DataFrame({'Avg_word': np.mean(new_word), 
                         'SD_word': np.std(new_word),
                         'Avg_sent': np.mean(new_sent),
                         'SD_sent': np.std(new_sent)}, index=[0])

In [8]:
gpt3_stat = stat(gpt3['Generation'])
gpt3_stat

/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.9/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.9/dist-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.9/dist-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/usr/local/lib/python3.9/dist-packages/numpy/core/_methods.py:256: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,Avg_word,SD_word,Avg_sent,SD_sent
0,286.193452,58.189374,NaN,NaN


In [9]:
ctrl_stat = stat(ctrl['Generation'])
ctrl_stat

/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.9/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.9/dist-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.9/dist-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/usr/local/lib/python3.9/dist-packages/numpy/core/_methods.py:256: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,Avg_word,SD_word,Avg_sent,SD_sent
0,426.044747,51.671647,NaN,NaN


In [10]:
ctrl.to_csv('new_ctrl.csv')
gpt.to_csv('new_gpt.csv')
gpt2.to_csv('new_gpt2.csv')
grover.to_csv('new_grover.csv')
xlm.to_csv('new_xlm.csv')
xlnet.to_csv('new_xlnet.csv')
pplm.to_csv('new_pplm.csv')
human.to_csv('new_human.csv')
fair.to_csv('new_gpt3.csv')
instructgpt.to_csv('new_instructgpt.csv')


In [11]:
new_gpt3 = pd.read_csv('new_gpt3.csv')
new_gpt3.head()

,Unnamed: 0,Generation,label
0,0,latest headlines on cnn business cnn businessn...,grover
1,1,china wants to take a victory lap over its han...,grover
2,2,coronavirus disinformation creates challenges ...,grover
3,3,china coronavirus eating wild animals made ill...,grover
4,4,chinas economy could shrink for the first time...,grover


In [12]:
generate = pd.concat([ctrl, gpt, gpt2, grover, xlm, xlnet, pplm, human, fair, gpt3, instructgpt])
generate
generate['label'].unique()

array(['ctrl', 'gpt', 'gpt2', 'grover', 'xlm', 'xlnet', 'pplm', 'human',
       'gpt3', 'instructgpt'], dtype=object)

In [13]:

label = []

for i in generate['label']:
    if i == 'ctrl':
        label.append(0)
    elif i == 'gpt':
        label.append(1)
    elif i == 'gpt2':
        label.append(2)
    elif i == 'grover':
        label.append(3)
    elif i == 'xlm':
        label.append(4)
    elif i == 'xlnet':
        label.append(5)
    elif i == 'pplm':
        label.append(6)
    elif i == 'human':
        label.append(7)
    elif i == 'fair':
        label.append(8)
    elif i=='gpt3':
        label.append(9)
    elif i=='instructgpt':
        label.append(10)
    else:
        label.append(7)


  
        
        

In [14]:
generation = pd.DataFrame({'text': generate['Generation'], 'class': generate['label']})
generation.to_csv('input.csv', index = False, header = True)
generation

,text,class
0,news latest headlines on cnn business n tldr t...,ctrl
1,news china wants to take a victory lap over it...,ctrl
2,news coronavirus disinformation creates challe...,ctrl
3,news china coronavirus eating wild animals mad...,ctrl
4,news chinas economy could shrink for the first...,ctrl
...,...,...
1061,how much of your body is your own your entire ...,instructgpt
1062,how do you keep a space station clean keeping ...,instructgpt
1063,the city where you pay a years rent up front t...,instructgpt
1064,the bbc news app gives you the best of bbc new...,instructgpt


In [1]:
import csv
import random

# Open the CSV file for reading
with open('input.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)

    # Skip the header row
    header = next(reader)

    # Shuffle the rows
    rows = list(reader)
    random.shuffle(rows)

# Open a new CSV file for writing
with open('shuffled_csv_file.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Write the header row
    writer.writerow(header)

    # Write the shuffled rows
    for row in rows:
        writer.writerow(row)


In [2]:
data = pd.read_csv('shuffled_csv_file.csv')
    # Print the column names to confirm that 'text' is present
print(data.columns)

NameError: ignored

In [4]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Download the required corpora and models
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Read in the labeled text data from CSV file
data = pd.read_csv('shuffled_csv_file.csv')

# Tokenize and tag the text data
tagged_data = []
for text in data['text']:
    tokens = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    tagged_text = ' '.join([tag[0] + '/' + tag[1] for tag in pos_tags])
    tagged_data.append(tagged_text)

# Split the tagged data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tagged_data, data['class'], test_size=0.3, random_state=42)

# Create a count vectorizer to convert the text data into numerical features
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train a MultinomialNB model on the vectorized data
model = MultinomialNB()
model.fit(X_train_vec, y_train)

# Make predictions on the test set and compute accuracy
y_pred = model.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Accuracy: 0.6611711199545196
